In [ ]:
!nvidia-smi

In [ ]:
#%pip install --extra-index-url=https://pypi.nvidia.com cudf-cu12
#%pip install cucim-cu12 cupy-cuda12x

In [1]:

import pandas as pd
import tensorflow as tf

import glob
import re

2024-11-17 00:45:30.421414: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-17 00:45:30.500550: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-17 00:45:30.519751: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-17 00:45:30.642504: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
%pip install fastparquet

In [2]:
base_path = '/home/workspace/geometry-classifier/data/'



In [ ]:
!du -h /home/workspace/geometry-classifier/data/tfrecord

In [3]:
import pandas as pd
import tensorflow as tf
import glob
import os

def serialize_example(name, image_data):
    """Serialize data into TFRecord format."""
    feature = {
        'name': tf.train.Feature(bytes_list=tf.train.BytesList(value=[name])),
        'image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image_data]))
    }
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

def parquet_to_tfrecord(parquet_file, tfrecord_file):
    """Convert a single Parquet file to a TFRecord file."""
    df = pd.read_csv(parquet_file).drop(columns=["id"])
    
    with tf.io.TFRecordWriter(tfrecord_file, options='GZIP') as writer:
        for _, row in df.iterrows():
            name = row['name'].astype('float32').tobytes()  # Convert uint8 name to bytes
            image_data = row.drop('name').to_numpy().astype('float32').tobytes()  # Flatten image data and convert to bytes
            example = serialize_example(name, image_data)
            writer.write(example)


def convert(file_type:str, files: list):
    i=0
    for parquet_file in files:
        tfrecord_file = f"{base_path}tfrecord/{file_type}{i}.tfrecord.gz"
        parquet_to_tfrecord(parquet_file, tfrecord_file)
        print(f"Converted {parquet_file} to {tfrecord_file}")
        i+=1


In [4]:
convert('train', glob.glob(f"{base_path}stratified/train*.csv"))
convert('test', glob.glob(f"{base_path}stratified/test*.csv"))
convert('val', glob.glob(f"{base_path}stratified/val*.csv"))

Converted /home/workspace/geometry-classifier/data/stratified/train_365.csv to /home/workspace/geometry-classifier/data/tfrecord/train0.tfrecord.gz
Converted /home/workspace/geometry-classifier/data/stratified/train_667.csv to /home/workspace/geometry-classifier/data/tfrecord/train1.tfrecord.gz
Converted /home/workspace/geometry-classifier/data/stratified/train_744.csv to /home/workspace/geometry-classifier/data/tfrecord/train2.tfrecord.gz
Converted /home/workspace/geometry-classifier/data/stratified/train_787.csv to /home/workspace/geometry-classifier/data/tfrecord/train3.tfrecord.gz
Converted /home/workspace/geometry-classifier/data/stratified/train_694.csv to /home/workspace/geometry-classifier/data/tfrecord/train4.tfrecord.gz
Converted /home/workspace/geometry-classifier/data/stratified/train_585.csv to /home/workspace/geometry-classifier/data/tfrecord/train5.tfrecord.gz
Converted /home/workspace/geometry-classifier/data/stratified/train_374.csv to /home/workspace/geometry-classifi